In [1]:
import numpy as np
import scipy.signal as sp
import pandas as pd

In [2]:
n=100

XCF_roisize=120
XCF_mesh=50
data_fill=[i for i in range(0,100)]
red_roisize=len(data_fill)


In [3]:
# im1_df=pd.read_csv('/Users/tom/Documents/GitHub/Fourier_XCF_speedup/im1')
# im2_df=pd.read_csv('/Users/tom/Documents/GitHub/Fourier_XCF_speedup/im2')

# array1=im1_df.to_numpy()
# array2=im2_df.to_numpy()

In [4]:
array1=np.zeros((n,n))
array2=np.zeros((n,n))

for i in range(0,n):
    for j in range(0,n):
        array1[i][j]=2*i*i*(i-50)+5+7*j+2;

for i in range(0,n):
    for j in range(0,n):
        array2[i][j]=7*j*j*(j-5)+8*i;

In [5]:
ROI_ref=np.fft.fft2(array1)
ROI_test=np.fft.fft2(array2)

In [6]:
CC=np.zeros((XCF_roisize*2,XCF_roisize*2),dtype='complex')

ind1=int(XCF_roisize-np.floor(red_roisize/2))
ind2=int(XCF_roisize+np.floor((red_roisize-1)/2))+1

In [7]:
f=np.fft.fftshift(ROI_test*np.conj(ROI_ref))
CC[ind1:ind2,ind1:ind2]= f

In [8]:
CC

array([[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       ...,
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]])

In [9]:
CC=np.fft.ifft2(CC)
#CC[3,4]=1234+0j
CC

array([[ 4.51013605e+14+1.85978495e-05j, -4.51013010e+14+1.92176896e+08j,
         4.51000135e+14-9.94780816e+07j, ...,
        -4.50937355e+14+1.40683252e+08j,  4.50967095e+14+9.94780816e+07j,
        -4.50996358e+14-1.92176896e+08j],
       [-4.51010219e+14+3.28683094e+07j,  4.51009624e+14-2.25045206e+08j,
        -4.50996748e+14+1.32346391e+08j, ...,
         4.50933968e+14-1.73551562e+08j, -4.50963709e+14-6.66097722e+07j,
         4.50992971e+14+1.59308587e+08j],
       [ 4.51004795e+14-1.70138889e+07j, -4.51004200e+14+2.09190785e+08j,
         4.50991325e+14-1.16491970e+08j, ...,
        -4.50928545e+14+1.57697141e+08j,  4.50958285e+14+8.24641927e+07j,
        -4.50987548e+14-1.75163007e+08j],
       ...,
       [-4.51010295e+14+2.40612724e+07j,  4.51009700e+14-2.16238169e+08j,
        -4.50996824e+14+1.23539354e+08j, ...,
         4.50934044e+14-1.64744525e+08j, -4.50963785e+14-7.54168092e+07j,
         4.50993047e+14+1.68115624e+08j],
       [ 4.51012201e+14+1.70138889e+07j, -4.

In [10]:
chi=np.amax(CC)
loc=np.argmax(CC)
rloc,cloc=np.unravel_index(loc,CC.shape)
rlocsave=rloc
clocsave=cloc

In [11]:
XCF_roisize2=2*XCF_roisize
if rloc > XCF_roisize:
    row_shift = rloc - XCF_roisize2
else:
    row_shift = rloc

if cloc > XCF_roisize:
    col_shift = cloc - XCF_roisize2
else:
    col_shift = cloc

row_shift = row_shift/2
col_shift = col_shift/2

In [12]:
row_shift

0.0

In [13]:
col_shift

0.0

In [14]:
#DFT computation
row_shift = round(row_shift*XCF_mesh)/XCF_mesh
col_shift = round(col_shift*XCF_mesh)/XCF_mesh
dftshift = np.floor(np.ceil(XCF_mesh*1.5)/2)

#matrix multiply DFT around current shift estimate
roff=dftshift-row_shift*XCF_mesh
coff=dftshift-col_shift*XCF_mesh

In [15]:
row_shift

0.0

In [16]:
#computer kernels and obtain DFT by matrix products
prefac=-1j*2*np.pi/(XCF_roisize*XCF_mesh)

In [17]:
prefac

-0.0010471975511965976j

In [18]:
c1=[i for i in range(0,(XCF_roisize))]
c_i=np.fft.ifftshift(c1).T - np.floor(XCF_roisize/2)
c_i=c_i[data_fill]
c_i=np.expand_dims(c_i,1)

r_i=np.fft.ifftshift(c1) - np.floor(XCF_roisize/2)
r_i=r_i[data_fill]
r_i=np.expand_dims(r_i,0)

In [19]:
r_i

array([[  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
         11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
         22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
         33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
         44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
         55.,  56.,  57.,  58.,  59., -60., -59., -58., -57., -56., -55.,
        -54., -53., -52., -51., -50., -49., -48., -47., -46., -45., -44.,
        -43., -42., -41., -40., -39., -38., -37., -36., -35., -34., -33.,
        -32., -31., -30., -29., -28., -27., -26., -25., -24., -23., -22.,
        -21.]])

In [20]:
m=np.array([i for i in range(0,int(np.ceil(XCF_mesh*1.5)))])
m1=np.expand_dims(m,0)
m2=np.expand_dims(m,1)

In [21]:
import numexpr as ne

arg1=(c_i)@(m1-coff)
arg2=(m2-roff)@(r_i)

kernc=ne.evaluate('exp(prefac*arg1)')
kernr=ne.evaluate('exp(prefac*arg2)')
kern = ROI_ref*ne.evaluate('conj(ROI_test)')
arg3=kernr@kern@kernc
CC2 = ne.evaluate('conj(arg3)')

In [22]:
prefac

-0.0010471975511965976j

In [23]:
(kern@kernc)

array([[2.59607032e+19-8.65591678e+15j, 2.59611595e+19-8.56414549e+15j,
        2.59616127e+19-8.46305071e+15j, ...,
        2.59718754e+19+1.24027452e+16j, 2.59716538e+19+1.27193399e+16j,
        2.59714241e+19+1.30312520e+16j],
       [1.37079242e+15-1.27071037e+15j, 1.37079242e+15-1.27071037e+15j,
        1.37079242e+15-1.27071037e+15j, ...,
        1.37079242e+15-1.27071037e+15j, 1.37079242e+15-1.27071037e+15j,
        1.37079242e+15-1.27071037e+15j],
       [4.58680930e+14-1.58838642e+14j, 4.58680930e+14-1.58838642e+14j,
        4.58680930e+14-1.58838642e+14j, ...,
        4.58680930e+14-1.58838642e+14j, 4.58680930e+14-1.58838642e+14j,
        4.58680930e+14-1.58838642e+14j],
       ...,
       [2.13727656e+14+4.70631020e+13j, 2.13727656e+14+4.70631020e+13j,
        2.13727656e+14+4.70631020e+13j, ...,
        2.13727656e+14+4.70631020e+13j, 2.13727656e+14+4.70631020e+13j,
        2.13727656e+14+4.70631020e+13j],
       [4.58680930e+14+1.58838642e+14j, 4.58680930e+14+1.58838642e+1

In [24]:
CC2

array([[2.59659876e+19+1.07829656e+16j, 2.59664440e+19+1.06911943e+16j,
        2.59668972e+19+1.05900995e+16j, ...,
        2.59771599e+19-1.02756964e+16j, 2.59769383e+19-1.05922911e+16j,
        2.59767086e+19-1.09042033e+16j],
       [2.59660158e+19+1.07282549e+16j, 2.59664722e+19+1.06364837e+16j,
        2.59669254e+19+1.05353889e+16j, ...,
        2.59771881e+19-1.03304070e+16j, 2.59769665e+19-1.06470018e+16j,
        2.59767368e+19-1.09589139e+16j],
       [2.59660426e+19+1.06729775e+16j, 2.59664990e+19+1.05812062e+16j,
        2.59669521e+19+1.04801114e+16j, ...,
        2.59772148e+19-1.03856845e+16j, 2.59769933e+19-1.07022792e+16j,
        2.59767635e+19-1.10141913e+16j],
       ...,
       [2.59637424e+19+7.52187652e+15j, 2.59641987e+19+7.43010524e+15j,
        2.59646519e+19+7.32901045e+15j, ...,
        2.59749146e+19-1.35367854e+16j, 2.59746930e+19-1.38533802e+16j,
        2.59744633e+19-1.41652923e+16j],
       [2.59636637e+19+7.51490002e+15j, 2.59641201e+19+7.42312874e+1

In [25]:
#locate maximum and map back to original pixel grid
CCmax=np.abs(np.amax(CC2))
loc=np.argmax(CC2)
loc1,loc2=np.unravel_index(loc,CC2.shape)

rloc=loc1-dftshift-1
cloc=loc2-dftshift-1

row_shift=row_shift+rloc/XCF_mesh
col_shift=col_shift+cloc/XCF_mesh
row_shift

-0.4

In [26]:
#normalise output
bf1=np.sum(ROI_test.flatten()*np.conjugate(ROI_test.flatten()))
bf2=np.sum(ROI_ref.flatten()*np.conjugate(ROI_ref.flatten()))

In [27]:
print(CCmax/np.sqrt(float(bf1*bf2)))
print(row_shift)
print(col_shift)

0.335767238343101
-0.4
0.26
0
0


/Users/tom/opt/anaconda3/envs/tpm/lib/python3.7/site-packages/ipykernel_launcher.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  """Entry point for launching an IPython kernel.
